In [1]:
import gym
import gym_road_interactions
from gym_road_interactions.envs.neighborhood_v4.neighborhood_env_v4_utils import calculate_poly_distance
import time
import logging
import sys, os, copy
import pdb, math
import glob
import copy, pickle
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import torch

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

# Logging function
def log(fname, s):
    f = open(fname, 'a')
    f.write(f'{str(datetime.now())}: {s}\n')
    f.close()

In [3]:
# dummy env_config for initializing the environment
ni_ep_len_dict = {1: 200, 2:1000, 3:1000}
ni = 1
na = 10
ed = 2
date = '0322'
env_config = {# parametric state
              'use_global_frame': False, # whether to use global frame for the state vector
              'normalize': True, # whether to normalize the state vector
              'include_ttc': True,
              'include_future_waypoints': 10,
              'num_history_states': 0, # this number will also be used in stalemate penalty!
              'include_polygon_dist': True, # if true, include sigmoid(polygon distance between ego and agent)
              'include_agents_within_range': 10.0, # the range within which agents will be included.
              'agent_state_dim': 6*(0+1)+4+10*2+1, # related to 'include_ttc', 'include_polygon_dist', 'include_future_waypoints''
              # num agents (change training env_config with this)
              'num_other_agents': na,
              'max_num_other_agents': 25, # could only be 25, 40, 60 (one of the upper limits)
              # agent behavior
              'agent_stochastic_stop': False, # (check) whether the other agent can choose to stop for ego with a certain probability
              'agent_shuffle_ids': True, # (check) if yes, the id of other agents will be shuffled during reset
              'rl_agent_configs' : [('',0.0)], # format: [(model_path, ratio_of_num_other_agents)]
              # path
              'ego_num_intersections_in_path': ni, # (check)
              'ego_expand_path_depth': ed, # (check) the number of extending lanes from center intersection in ego path
              'expanded_lane_set_depth': 1, # (check)
              'single_intersection_type': 'mix', # (check) ['t-intersection', 'roundabout', 'mix']
              'c1': 2, # (check) 0: left/right turn at t4, 1: all possible depth-1 paths at t4, 2: all possible depth-1 paths at random intersection
              # training settings
              'gamma': 0.99,
              'max_episode_timesteps': ni_ep_len_dict[ni],
              # NOTE: if you change velocity_coeff, you have to change the whole dataset 
              # (interaction set, collision set, etc) bc they are based on b1 and locations are 
              # based on v_desired
              'ego_velocity_coeff': (2.7, 8.3), # (check) (w, b). v_desired = w * b1 + b # experiments change this one
              'agent_velocity_coeff': (2.7, 8.3), # (check) (w, b). v_desired = w * b1 + b # this is fixed!
              # reward = w * b1 + b # Be careful with signs!
              'reward': 'default_fad', # [default, default_ttc, simplified, default_fad]
              'time_penalty_coeff': (-1./20., -3./20.),
              'velocity_reward_coeff': (0.5, 1.5),
              'collision_penalty_coeff': (0.0, -40.0),
              'fad_penalty_coeff': 1.0,
              'timeout_penalty_coeff': (-5.0, -20.0),
              'stalemate_penalty_coeff': (-0.5, -1.5),
              # ego config
              'use_default_ego': False,
              'no_ego': False, # TODO: remember to turn this off if you want ego!
              # action noises
              'ego_action_noise': 0.0, # lambda of the poisson noise applied to ego action
              'agent_action_noise': 0.0 # lambda of the poisson noise applied to agent action
              } 
# training configs
train_configs = {# model
                 'model': 'DDQN', # [TwinDDQN, DDQN]
                 'gamma': env_config['gamma'],
                 'target_update': 100, # number of policy updates until target update
                 'max_buffer_size': 100000, # max size for buffer that saves state-action-reward transitions
                 'batch_size': 128,
                 'lrt': 2e-5,
                 'tau': 0.2,
                 'exploration_timesteps': 0,
                 'value_net': 'set_transformer', # [vanilla, deep_set, deeper_deep_set, set_transformer, social_attention]
                 # deep set
                 'pooling': 'mean', # ['mean', 'max']
                 # set transformer
                 'layer_norm' : False, # whether to use layer_norm in set transformer
                 'layers' : 1,
                 # 'train_every_timesteps': 4,
                 # epsilon decay for epsilon greedy
                 'eps_start': 1.0,
                 'eps_end': 0.01,
                 'eps_decay': 500,
                 # training 
                 'reward_threshold': 1000, # set impossible
                 'max_timesteps': 100000,
                 'train_every_episodes': -1, # if -1: train at every timestep. if n >= 1: train after every n timesteps
                 'save_every_timesteps': 5000,
                 'eval_every_episodes': 50, # if -1: don't eval
                 'log_every_timesteps': 100,
                 'record_every_episodes': 30,
                 'seed': 0,
                 'grad_clamp': False,
                 'moving_avg_window': 100,
                 'replay_collision_episode': 0, # if 0, start every new episode fresh
                 'replay_episode': 0, # if 0, start every new episode fresh
                 'collision_episode_ratio': 0.5, # the ratio of saved collision episodes used in both training and testing. if set to 0, then don't use saved episodes
                 'buffer_agent_states': False, # if True, include agent-centric states in replay buffer
                 # env
                 'agent_state_dim': env_config['agent_state_dim'], # state_dim of each agent
                 'state_dim': env_config['agent_state_dim'] * (env_config['num_other_agents']+1), # total state_dim
                 'action_dim': 2,
                 'env_action_dim': 1,
                 'max_episode_timesteps': ni_ep_len_dict[env_config['ego_num_intersections_in_path']]}

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
print(device)

env = gym.make('Neighborhood-v4')
env.set_env_config(env_config)
assert env_config['max_num_other_agents'] == 25
env.set_train_config_and_device(train_configs, device)

cuda


In [43]:
# intersection_points = [[(lane, waypoint_idx)]]
# each list in the list is a list of INCOMING lanes intersecting at this point

intersection_points = [[('B6',1601), ('B10',1000)],
                       [('B5',1300)],
                       [('B1',1300)],
                       [('B10',200),('B11',800),('B7',800)],
                       [('B7',1601),('B9',600)],
                       [('B8',600),('B11',1000)],
                       [('B3',1300)],
                       [('E18',1120),('E11',360)],
                       [('E19',724)]]

# Interaction set with arrival perturbation

In [56]:
env = gym.make('Neighborhood-v4')
env.set_env_config(env_config)
assert env_config['max_num_other_agents'] == 25
env.set_train_config_and_device(train_configs, device)
env.log_name_ = 'out/interaction_set_creation.log'

test_set_init_dicts = []

## Setting 1

In [57]:
# == Setting 1 ==
w = 2.7
b = 8.3
# 0601: change to evenly divided
a_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
b_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
agent_arrival_noise = [np.random.randint(-4,-1), # [-4,-2]
                       4 - np.random.randint(0,3)] # [2,4]
ego_pred_depth = 2
agent_pred_depth = 3

# [{'ego': ego_init_dict, 
#   'agents': [agent_init_dict]}]
env_config['num_other_agents'] = 1

# 14 * 3 * 3 * 2 = 252
for p in range(len(intersection_points)):
    intersection_point_entry = intersection_points[p]
    print(f"intersection_point {p}...")
    # deciding who's A, who's B
    if len(intersection_point_entry) == 1:
        continue
    elif len(intersection_point_entry) == 2:
        permutations = [(0,1),(1,0)]
    elif len(intersection_point_entry) == 3:
        permutations = [(0,1),(1,0),(1,2),(2,1),(0,2),(2,0)]
        
    for permutation in permutations:
        print(f"permutation={permutation}")
        for b_b1_interval in b_b1_intervals:
            b_b1 = np.random.random() * (b_b1_interval[1] - b_b1_interval[0]) + b_b1_interval[0]
            b_v_desired = w * b_b1 + b
            print(f"b_b1: {b_b1}, b_v_desired: {b_v_desired}")
            for a_b1_interval in a_b1_intervals:
                a_b1 = np.random.random() * (a_b1_interval[1] - a_b1_interval[0]) + a_b1_interval[0]
                a_v_desired = w * a_b1 + b
                print(f"a_b1: {a_b1}, a_v_desired: {a_v_desired}")
                for arrival_noise in agent_arrival_noise:
                    print(f"arrival_noise: {arrival_noise}")
                    ego_path = [intersection_point_entry[permutation[0]][0]]
                    # ego_path.insert(0, env.lane_segments_[intersection_point_entry[permutation[0]][0]].predecessors[0])
                    for _ in range(ego_pred_depth):
                        ego_path.insert(0, env.lane_segments_[ego_path[0]].predecessors[0])
                    ego_path.append(env.lane_segments_[intersection_point_entry[permutation[0]][0]].successors[0])
                    ego_path.append(env.lane_segments_[ego_path[-1]].successors[0]) # ed=2
                    ego_init_dict = {'path': ego_path, 'curr_lane_id_order': ego_pred_depth, 
                                     'curr_waypoint_idx': intersection_point_entry[permutation[0]][1],
                                     'v_desired': a_v_desired, 'b1': a_b1}

                    agent_path = [intersection_point_entry[permutation[1]][0]]
                    # add (agent_pred_depth)-degree predecessors to agent_path to ensure there's enough space to retreat to
                    for d in range(agent_pred_depth):
                        # find the longest predecessor
                        preds = env.lane_segments_[agent_path[0]].predecessors
                        longest_pred_idx = 0
                        if len(preds) > 1:
                            longest_pred_idx = -1
                            longest_pred_length = -float('Inf')
                            for j in range(len(preds)):
                                if len(env.lane_segments_[preds[j]].centerline) > longest_pred_length:
                                    longest_pred_idx = j
                                    longest_pred_length = len(env.lane_segments_[preds[j]].centerline)
                        agent_path.insert(0, env.lane_segments_[agent_path[0]].predecessors[longest_pred_idx])

                    agent_path.append(env.lane_segments_[intersection_point_entry[permutation[1]][0]].successors[0])
                    agent_extending_path = env.generate_agent_extending_path(agent_path[-1]) # make sure agent keeps going
                    agent_path = agent_path + agent_extending_path[1:]
                    agent_init_dicts = [{'path': agent_path, 'curr_lane_id_order': agent_pred_depth, 
                                         'curr_waypoint_idx': intersection_point_entry[permutation[1]][1], 
                                         'v_desired': b_v_desired, 'b1': b_b1}]
                    agents_dict = env.convert_dict_to_agents(ego_init_dict, agent_init_dicts)
                    assert len(agent_init_dicts) == env_config['num_other_agents']
                    state = env.reset(use_saved_agents=agents_dict)

                    cnt = 0
                    target_cnt = 24
                    while (cnt < target_cnt):
                        env.agents_['0'].step(-1, env.dt_)
                        env.agents_['1'].force_action(-1, env.dt_)
                        cnt += 1
                        if (env.agents_['1'].closest_lane_id_order_ == 0 and env.agents_['1'].closest_waypoint_idx_ == 0):
                            raise Exception("agent 1 pushed out of path")
                        if (env.agents_['0'].closest_lane_id_order_ == 0 and env.agents_['0'].closest_waypoint_idx_ == 0):
                            raise Exception("agent 0 pushed out of path")


                    # agent arrival noise
                    if arrival_noise < 0:
                        for t in range(abs(arrival_noise)):
                            env.agents_['1'].force_action(-1, env.dt_)
                            if (env.agents_['1'].closest_lane_id_order_ == 0 and env.agents_['1'].closest_waypoint_idx_ == 0):
                                raise Exception("agent 1 pushed out of path")
                    else:
                        for t in range(arrival_noise):
                            env.agents_['1'].force_action(1, env.dt_)

                    test_set_ego_dict = {'path': ego_init_dict['path'], 
                                         'curr_lane_id_order': env.agents_['0'].closest_lane_id_order_, 
                                         'curr_waypoint_idx': env.agents_['0'].closest_waypoint_idx_,
                                         'v_desired': env.agents_['0'].v_desired_, 'b1': env.agents_['0'].b1_}
                    test_set_agent_dict = {'path': agent_init_dicts[0]['path'], 
                                           'curr_lane_id_order': env.agents_['1'].closest_lane_id_order_, 
                                           'curr_waypoint_idx': env.agents_['1'].closest_waypoint_idx_,
                                           'v_desired': env.agents_['1'].v_desired_, 'b1': env.agents_['1'].b1_}
                    test_set_init_dicts.append({'ego': test_set_ego_dict, 
                                                'agents': [test_set_agent_dict], 
                                                'env_config': copy.deepcopy(env_config),
                                                'setting': 1})


intersection_point 0...
permutation=(0, 1)
b_b1: -0.5360330541433782, b_v_desired: 6.85271075381288
a_b1: -0.8477269725517983, a_v_desired: 6.011137174110145
arrival_noise: -2
arrival_noise: 3
a_b1: -0.25546268330543265, a_v_desired: 7.610250755075333
arrival_noise: -2
arrival_noise: 3
a_b1: 0.615487156328545, a_v_desired: 9.961815322087073
arrival_noise: -2
arrival_noise: 3
b_b1: -0.11456373758187494, b_v_desired: 7.990677908528938
a_b1: -0.4707957591574997, a_v_desired: 7.028851450274751
arrival_noise: -2
arrival_noise: 3
a_b1: -0.11224377978636307, a_v_desired: 7.99694179457682
arrival_noise: -2
arrival_noise: 3
a_b1: 0.6965706712978625, a_v_desired: 10.18074081250423
arrival_noise: -2
arrival_noise: 3
b_b1: 0.8448352619566772, b_v_desired: 10.581055207283029
a_b1: -0.3343697686990117, a_v_desired: 7.397201624512669
arrival_noise: -2
arrival_noise: 3
a_b1: -0.22893086404032875, a_v_desired: 7.6818866670911135
arrival_noise: -2
arrival_noise: 3
a_b1: 0.4762455641406491, a_v_desired: 

arrival_noise: 3
intersection_point 4...
permutation=(0, 1)
b_b1: -0.4909482064436187, b_v_desired: 6.97443984260223
a_b1: -0.9966511936485328, a_v_desired: 5.609041777148962
arrival_noise: -2
arrival_noise: 3
a_b1: 0.2444660586511112, a_v_desired: 8.960058358358001
arrival_noise: -2
arrival_noise: 3
a_b1: 0.8249117263665473, a_v_desired: 10.527261661189678
arrival_noise: -2
arrival_noise: 3
b_b1: 0.3035605216019955, b_v_desired: 9.119613408325389
a_b1: -0.7482560600264179, a_v_desired: 6.279708637928673
arrival_noise: -2
arrival_noise: 3
a_b1: -0.18990422807375804, a_v_desired: 7.787258584200854
arrival_noise: -2
arrival_noise: 3
a_b1: 0.5160634312515129, a_v_desired: 9.693371264379085
arrival_noise: -2
arrival_noise: 3
b_b1: 0.8814058507784688, b_v_desired: 10.679795797101868
a_b1: -0.9928998010632265, a_v_desired: 5.619170537129289
arrival_noise: -2
arrival_noise: 3
a_b1: 0.16109666605912648, a_v_desired: 8.734960998359643
arrival_noise: -2
arrival_noise: 3
a_b1: 0.6456712701574205,

In [58]:
print(len(test_set_init_dicts))

252


## Setting 2

In [59]:
# == Setting 2 ==
w = 2.7
b = 8.3
a_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
b_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
c_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
agent_arrival_noise = [np.random.randint(-4,-1), # [-4,-1)
                       4 - np.random.randint(0,3)] # (1,4]
ego_pred_depth = 2
agent_pred_depth = 5

env_config['num_other_agents'] = 2

# 14 * 3 * 3 * 3 * 2 = 756
for p in range(len(intersection_points)):
    intersection_point_entry = intersection_points[p]
    print(f"intersection_point {p}...")
    # deciding who's A, who's B
    if len(intersection_point_entry) == 1:
        continue
    elif len(intersection_point_entry) == 2:
        permutations = [(0,1),(1,0)]
    elif len(intersection_point_entry) == 3:
        permutations = [(0,1),(1,0),(1,2),(2,1),(0,2),(2,0)]
    
    for permutation in permutations:
        print(f"permutation={permutation}")
        for a_b1_interval in a_b1_intervals:
            a_b1 = np.random.random() * (a_b1_interval[1] - a_b1_interval[0]) + a_b1_interval[0]
            a_v_desired = w * a_b1 + b
            print(f"a_b1: {a_b1}, a_v_desired: {a_v_desired}")
            
            for b_b1_interval in b_b1_intervals:
                b_b1 = np.random.random() * (b_b1_interval[1] - b_b1_interval[0]) + b_b1_interval[0]
                b_v_desired = w * b_b1 + b
                print(f"b_b1: {b_b1}, b_v_desired: {b_v_desired}")
            
                for c_b1_interval in c_b1_intervals:
                    c_b1 = np.random.random() * (c_b1_interval[1] - c_b1_interval[0]) + c_b1_interval[0]
                    c_v_desired = w * c_b1 + b
                    print(f"c_b1: {c_b1}, c_v_desired: {c_v_desired}")
                
                    for arrival_noise in agent_arrival_noise:
                        print(f"arrival_noise: {arrival_noise}")
                        ego_path = [intersection_point_entry[permutation[0]][0]]
                        for _ in range(ego_pred_depth):
                            ego_path.insert(0, env.lane_segments_[ego_path[0]].predecessors[0])
                        ego_path.append(env.lane_segments_[intersection_point_entry[permutation[0]][0]].successors[0])
                        ego_path.append(env.lane_segments_[ego_path[-1]].successors[0]) # ed=2
                        ego_init_dict = {'path': ego_path, 'curr_lane_id_order': ego_pred_depth, 
                                         'curr_waypoint_idx': intersection_point_entry[permutation[0]][1],
                                         'v_desired': a_v_desired, 'b1': a_b1}

                        agent_path = [intersection_point_entry[permutation[1]][0]]
                        # add 4-degree predecessors to agent_path to ensure there's enough space to retreat to
                        for d in range(agent_pred_depth):
                            # find the longest predecessor
                            preds = env.lane_segments_[agent_path[0]].predecessors
                            longest_pred_idx = 0
                            if len(preds) > 1:
                                longest_pred_idx = -1
                                longest_pred_length = -float('Inf')
                                for j in range(len(preds)):
                                    if len(env.lane_segments_[preds[j]].centerline) > longest_pred_length:
                                        longest_pred_idx = j
                                        longest_pred_length = len(env.lane_segments_[preds[j]].centerline)
                            agent_path.insert(0, env.lane_segments_[agent_path[0]].predecessors[longest_pred_idx])

                        agent_path.append(env.lane_segments_[intersection_point_entry[permutation[1]][0]].successors[0])
                        agent_extending_path = env.generate_agent_extending_path(agent_path[-1]) # make sure agent keeps going
                        agent_path = agent_path + agent_extending_path[1:]
                        agent_init_dicts = [{'path': agent_path, 'curr_lane_id_order': agent_pred_depth, 
                                             'curr_waypoint_idx': intersection_point_entry[permutation[1]][1], 
                                             'v_desired': b_v_desired, 'b1': b_b1},
                                            {'path': agent_path, 'curr_lane_id_order': agent_pred_depth, 
                                             'curr_waypoint_idx': intersection_point_entry[permutation[1]][1], 
                                             'v_desired': c_v_desired, 'b1': c_b1}]
                        agents_dict = env.convert_dict_to_agents(ego_init_dict, agent_init_dicts)
                        assert len(agent_init_dicts) == env_config['num_other_agents']
                        state = env.reset(use_saved_agents=agents_dict)

                        cnt = 0
                        target_cnt = 24
                        while (cnt < target_cnt):
                            env.agents_['0'].step(-1, env.dt_)
                            env.agents_['1'].force_action(-1, env.dt_)
                            cnt += 1
                            if (env.agents_['1'].closest_lane_id_order_ == 0 and env.agents_['1'].closest_waypoint_idx_ == 0):
                                raise Exception("agent 1 pushed out of path")
                            if (env.agents_['0'].closest_lane_id_order_ == 0 and env.agents_['0'].closest_waypoint_idx_ == 0):
                                raise Exception("agent 0 pushed out of path")

                        # agent arrival noise
                        if arrival_noise < 0:
                            for t in range(abs(arrival_noise)):
                                env.agents_['1'].force_action(-1, env.dt_)
                                if (env.agents_['1'].closest_lane_id_order_ == 0 and env.agents_['1'].closest_waypoint_idx_ == 0):
                                    raise Exception("agent 1 pushed out of path")
                        else:
                            for t in range(arrival_noise):
                                env.agents_['1'].force_action(1, env.dt_)

                        # now find C's position
                        # change C dict to match B location
                        agent_init_dicts[1]['curr_lane_id_order'] = env.agents_['1'].closest_lane_id_order_
                        agent_init_dicts[1]['curr_waypoint_idx'] = env.agents_['1'].closest_waypoint_idx_
                        agent_c = env.convert_dict_to_agent(agent_init_dicts[1], False, '2')
                        env.agents_['2'] = agent_c

                        while not (calculate_poly_distance(env.agents_['1'], env.agents_['2']) >= env.agents_['2'].fad_distance()):
                            env.agents_['2'].force_action(-1, env.dt_)
                            if (env.agents_['2'].closest_lane_id_order_ == 0 and env.agents_['2'].closest_waypoint_idx_ == 0):
                                raise Exception("agent 2 pushed out of path")

                        # save the 3 agents info
                        test_set_ego_dict = {'path': ego_init_dict['path'], 
                                             'curr_lane_id_order': env.agents_['0'].closest_lane_id_order_, 
                                             'curr_waypoint_idx': env.agents_['0'].closest_waypoint_idx_,
                                             'v_desired': env.agents_['0'].v_desired_, 
                                             'b1': env.agents_['0'].b1_}
                        test_set_agent_dict_list = []
                        for i in range(len(agent_init_dicts)):
                            test_set_agent_dict = {'path': agent_init_dicts[i]['path'], 
                                               'curr_lane_id_order': env.agents_[str(i+1)].closest_lane_id_order_, 
                                               'curr_waypoint_idx': env.agents_[str(i+1)].closest_waypoint_idx_,
                                               'agg_level': env.agents_[str(i+1)].agg_level_,
                                               'v_desired': env.agents_[str(i+1)].v_desired_,
                                               'b1': env.agents_[str(i+1)].b1_,}
                            test_set_agent_dict_list.append(test_set_agent_dict)

                        test_set_init_dicts.append({'ego': test_set_ego_dict, 'agents': test_set_agent_dict_list, 
                                                    'env_config': copy.deepcopy(env_config), 'setting': 2})


intersection_point 0...
permutation=(0, 1)
a_b1: -0.5289125754839397, a_v_desired: 6.871936046193364
b_b1: -0.7398887160306418, b_v_desired: 6.302300466717268
c_b1: -0.46867424685571535, c_v_desired: 7.034579533489569
arrival_noise: -3
arrival_noise: 3
c_b1: 0.15797535306177424, c_v_desired: 8.726533453266791
arrival_noise: -3
arrival_noise: 3
c_b1: 0.8848755320986681, c_v_desired: 10.689163936666404
arrival_noise: -3
arrival_noise: 3
b_b1: 0.27177876854292277, b_v_desired: 9.033802675065893
c_b1: -0.6757175351212761, c_v_desired: 6.475562655172555
arrival_noise: -3
arrival_noise: 3
c_b1: -0.12728425023933718, c_v_desired: 7.95633252435379
arrival_noise: -3
arrival_noise: 3
c_b1: 0.8182903620922664, c_v_desired: 10.50938397764912
arrival_noise: -3
arrival_noise: 3
b_b1: 0.8505837490870245, b_v_desired: 10.596576122534966
c_b1: -0.8158939447539895, c_v_desired: 6.097086349164229
arrival_noise: -3
arrival_noise: 3
c_b1: 0.069239840422673, c_v_desired: 8.486947569141218
arrival_noise: -3


c_b1: -0.12395632285885638, c_v_desired: 7.965317928281088
arrival_noise: -3
arrival_noise: 3
c_b1: 0.7525149766098402, c_v_desired: 10.33179043684657
arrival_noise: -3
arrival_noise: 3
a_b1: 0.81836013628403, a_v_desired: 10.509572367966882
b_b1: -0.700450491281615, b_v_desired: 6.40878367353964
c_b1: -0.887454654045651, c_v_desired: 5.903872434076742
arrival_noise: -3
arrival_noise: 3
c_b1: -0.05829316517165806, c_v_desired: 8.142608454036523
arrival_noise: -3
arrival_noise: 3
c_b1: 0.7494448893362473, c_v_desired: 10.323501201207868
arrival_noise: -3
arrival_noise: 3
b_b1: 0.2533353911994361, b_v_desired: 8.984005556238479
c_b1: -0.38379887203488594, c_v_desired: 7.263743045505809
arrival_noise: -3
arrival_noise: 3
c_b1: 0.07056681030574224, c_v_desired: 8.490530387825505
arrival_noise: -3
arrival_noise: 3
c_b1: 0.5397598002145568, c_v_desired: 9.757351460579304
arrival_noise: -3
arrival_noise: 3
b_b1: 0.6370172308019058, b_v_desired: 10.019946523165146
c_b1: -0.4107762014436046, c_

arrival_noise: 3
c_b1: 0.3986131223947612, c_v_desired: 9.376255430465855
arrival_noise: -3
arrival_noise: 3
b_b1: 0.5323422247188452, b_v_desired: 9.737324006740883
c_b1: -0.8488122589243117, c_v_desired: 6.008206900904359
arrival_noise: -3
arrival_noise: 3
c_b1: 0.03810418092806239, c_v_desired: 8.40288128850577
arrival_noise: -3
arrival_noise: 3
c_b1: 0.5189090836742529, c_v_desired: 9.701054525920483
arrival_noise: -3
arrival_noise: 3
a_b1: -0.3299353336833004, a_v_desired: 7.409174599055089
b_b1: -0.6820316443437804, b_v_desired: 6.4585145602717935
c_b1: -0.9499586543581898, c_v_desired: 5.735111633232888
arrival_noise: -3
arrival_noise: 3
c_b1: 0.14555038986889968, c_v_desired: 8.69298605264603
arrival_noise: -3
arrival_noise: 3
c_b1: 0.6944380674999062, c_v_desired: 10.174982782249748
arrival_noise: -3
arrival_noise: 3
b_b1: 0.22355150134925622, b_v_desired: 8.903589053642992
c_b1: -0.47568918538965843, c_v_desired: 7.015639199447923
arrival_noise: -3
arrival_noise: 3
c_b1: -0.0

c_b1: 0.5950001927084285, c_v_desired: 9.906500520312758
arrival_noise: -3
arrival_noise: 3
b_b1: 0.4690566077483771, b_v_desired: 9.566452840920618
c_b1: -0.6053303615907912, c_v_desired: 6.6656080237048645
arrival_noise: -3
arrival_noise: 3
c_b1: -0.15046772146985993, c_v_desired: 7.893737152031379
arrival_noise: -3
arrival_noise: 3
c_b1: 0.9652838988086783, c_v_desired: 10.906266526783432
arrival_noise: -3
arrival_noise: 3
intersection_point 4...
permutation=(0, 1)
a_b1: -0.4582630445458924, a_v_desired: 7.0626897797260915
b_b1: -0.3349124976924778, b_v_desired: 7.395736256230311
c_b1: -0.8572271081825846, c_v_desired: 5.985486807907022
arrival_noise: -3
arrival_noise: 3
c_b1: -0.23355219999170235, c_v_desired: 7.669409060022405
arrival_noise: -3
arrival_noise: 3
c_b1: 0.7329535540757884, c_v_desired: 10.27897459600463
arrival_noise: -3
arrival_noise: 3
b_b1: -0.20817680618490472, b_v_desired: 7.737922623300758
c_b1: -0.5025100116913717, c_v_desired: 6.943222968433297
arrival_noise:

b_b1: -0.0561802921869019, b_v_desired: 8.148313211095365
c_b1: -0.4404061834956522, c_v_desired: 7.11090330456174
arrival_noise: -3
arrival_noise: 3
c_b1: -0.30310499972622734, c_v_desired: 7.481616500739187
arrival_noise: -3
arrival_noise: 3
c_b1: 0.5343619647795168, c_v_desired: 9.742777304904696
arrival_noise: -3
arrival_noise: 3
b_b1: 0.849673507954344, b_v_desired: 10.59411847147673
c_b1: -0.6422272153798825, c_v_desired: 6.565986518474318
arrival_noise: -3
arrival_noise: 3
c_b1: 0.3083866086535982, c_v_desired: 9.132643843364717
arrival_noise: -3
arrival_noise: 3
c_b1: 0.8047802949158434, c_v_desired: 10.472906796272778
arrival_noise: -3
arrival_noise: 3
a_b1: 0.9419274975319034, a_v_desired: 10.84320424333614
b_b1: -0.9513408223476609, b_v_desired: 5.7313797796613155
c_b1: -0.4829225584744161, c_v_desired: 6.996109092119077
arrival_noise: -3
arrival_noise: 3
c_b1: -0.1454524980175123, c_v_desired: 7.9072782553527174
arrival_noise: -3
arrival_noise: 3
c_b1: 0.5579285239545902, c

arrival_noise: 3
permutation=(1, 0)
a_b1: -0.33685200231527024, a_v_desired: 7.390499593748771
b_b1: -0.512731959533816, b_v_desired: 6.915623709258697
c_b1: -0.9550267523452879, c_v_desired: 5.7214277686677235
arrival_noise: -3
arrival_noise: 3
c_b1: 0.16271787893314865, c_v_desired: 8.739338273119502
arrival_noise: -3
arrival_noise: 3
c_b1: 0.36935532510863867, c_v_desired: 9.297259377793326
arrival_noise: -3
arrival_noise: 3
b_b1: -0.010596311410193404, b_v_desired: 8.27138995919248
c_b1: -0.8413573715500443, c_v_desired: 6.028335096814881
arrival_noise: -3
arrival_noise: 3
c_b1: -0.06645447800261006, c_v_desired: 8.120572909392953
arrival_noise: -3
arrival_noise: 3
c_b1: 0.4247805380607498, c_v_desired: 9.446907452764025
arrival_noise: -3
arrival_noise: 3
b_b1: 0.6058278998351032, b_v_desired: 9.93573532955478
c_b1: -0.5053389368397376, c_v_desired: 6.9355848705327094
arrival_noise: -3
arrival_noise: 3
c_b1: -0.1666297958056548, c_v_desired: 7.850099551324733
arrival_noise: -3
arri

In [60]:
print(len(test_set_init_dicts))

1008


## Setting 3

In [61]:
# == Setting 3 ==
w = 2.7
b = 8.3
a_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
b_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
c_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
b_arrival_noises = [np.random.randint(-4,-1), # [-4,-1)
                       4 - np.random.randint(0,3)] # (1,4]
c_arrival_noises = [np.random.randint(-4,-1), # [-4,-1)
                       4 - np.random.randint(0,3)] # (1,4]
ego_pred_depth = 2
agent_pred_depth = 2

env_config['num_other_agents'] = 2

# 1 * 3 * 3 * 3 * 3 * 2 * 2 = 324
for p in range(len(intersection_points)):
    intersection_point_entry = intersection_points[p]
    print(f"intersection_point {p}...")
    # deciding who's A, who's B
    if len(intersection_point_entry) < 3:
        continue
    else:
        permutations = [(0,1,2),(1,0,2),(2,0,1)]
    
    for permutation in permutations:
        print(f"permutation={permutation}")
        for a_b1_interval in a_b1_intervals:
            a_b1 = np.random.random() * (a_b1_interval[1] - a_b1_interval[0]) + a_b1_interval[0]
            a_v_desired = w * a_b1 + b
            print(f"a_b1: {a_b1}, a_v_desired: {a_v_desired}")
            
            for b_b1_interval in b_b1_intervals:
                b_b1 = np.random.random() * (b_b1_interval[1] - b_b1_interval[0]) + b_b1_interval[0]
                b_v_desired = w * b_b1 + b
                print(f"b_b1: {b_b1}, b_v_desired: {b_v_desired}")
            
                for c_b1_interval in c_b1_intervals:
                    c_b1 = np.random.random() * (c_b1_interval[1] - c_b1_interval[0]) + c_b1_interval[0]
                    c_v_desired = w * c_b1 + b
                    print(f"c_b1: {c_b1}, c_v_desired: {c_v_desired}")
                    
                    bc_b1 = [b_b1, c_b1]
                    bc_v_desired = [b_v_desired, c_v_desired]
                    
                    for b_arrival_noise in b_arrival_noises:
                        print(f"b_arrival_noise: {b_arrival_noise}")
                        for c_arrival_noise in c_arrival_noises:
                            print(f"c_arrival_noise: {c_arrival_noise}")
                            curr_arrival_noises = [b_arrival_noise, c_arrival_noise]
                            ego_path = [intersection_point_entry[permutation[0]][0]]
                            for _ in range(ego_pred_depth):
                                ego_path.insert(0, env.lane_segments_[ego_path[0]].predecessors[0])
                            ego_path.append(env.lane_segments_[intersection_point_entry[permutation[0]][0]].successors[0])
                            ego_path.append(env.lane_segments_[ego_path[-1]].successors[0]) # ed=2
                            ego_init_dict = {'path': ego_path, 'curr_lane_id_order': ego_pred_depth, 
                                             'curr_waypoint_idx': intersection_point_entry[permutation[0]][1],
                                             'v_desired': a_v_desired, 'b1': a_b1}

                            agent_init_dicts = []
                            for j in range(1,3):
                                agent_path = [intersection_point_entry[permutation[j]][0]]
                                # add 4-degree predecessors to agent_path to ensure there's enough space to retreat to
                                for d in range(agent_pred_depth):
                                    # find the longest predecessor
                                    preds = env.lane_segments_[agent_path[0]].predecessors
                                    longest_pred_idx = 0
                                    if len(preds) > 1:
                                        longest_pred_idx = -1
                                        longest_pred_length = -float('Inf')
                                        for j in range(len(preds)):
                                            if len(env.lane_segments_[preds[j]].centerline) > longest_pred_length:
                                                longest_pred_idx = j
                                                longest_pred_length = len(env.lane_segments_[preds[j]].centerline)
                                    agent_path.insert(0, env.lane_segments_[agent_path[0]].predecessors[longest_pred_idx])

                                agent_path.append(env.lane_segments_[intersection_point_entry[permutation[j]][0]].successors[0])
                                agent_extending_path = env.generate_agent_extending_path(agent_path[-1]) # make sure agent keeps going
                                agent_path = agent_path + agent_extending_path[1:]
                                agent_init_dicts.append({'path': agent_path, 'curr_lane_id_order': agent_pred_depth, 
                                                         'curr_waypoint_idx': intersection_point_entry[permutation[j]][1], 
                                                         'v_desired': bc_v_desired[j-1], 'b1': bc_b1[j-1]})

                            agents_dict = env.convert_dict_to_agents(ego_init_dict, agent_init_dicts)
                            assert len(agent_init_dicts) == env_config['num_other_agents']
                            state = env.reset(use_saved_agents=agents_dict)

                            cnt = 0
                            target_cnt = 24
                            while (cnt < target_cnt):
                                env.agents_['0'].step(-1, env.dt_)
                                env.agents_['1'].force_action(-1, env.dt_)
                                env.agents_['2'].force_action(-1, env.dt_)
                                cnt += 1
                                # check if any of the agents gets pushed out of path
                                for agent_id in range(env_config['num_other_agents'] + 1):
                                    agent_id_str = str(agent_id)
                                    if (env.agents_[agent_id_str].closest_lane_id_order_ == 0 and env.agents_[agent_id_str].closest_waypoint_idx_ == 0):
                                        raise Exception(f"agent {agent_id_str} pushed out of path")

                            # agent arrival noise
                            for j in range(1,3):
                                j_str = str(j)
                                if curr_arrival_noises[j-1] < 0:
                                    for t in range(abs(curr_arrival_noises[j-1])):
                                        env.agents_[j_str].force_action(-1, env.dt_)
                                        if (env.agents_[j_str].closest_lane_id_order_ == 0 and env.agents_[j_str].closest_waypoint_idx_ == 0):
                                            raise Exception(f"agent {j_str} pushed out of path")
                                else:
                                    for t in range(curr_arrival_noises[j-1]):
                                        env.agents_[j_str].force_action(1, env.dt_)

                            # save the 3 agents info
                            test_set_ego_dict = {'path': ego_init_dict['path'], 
                                                 'curr_lane_id_order': env.agents_['0'].closest_lane_id_order_, 
                                                 'curr_waypoint_idx': env.agents_['0'].closest_waypoint_idx_,
                                                 'v_desired': env.agents_['0'].v_desired_,
                                                 'b1': env.agents_['0'].b1_}
                            test_set_agent_dict_list = []
                            for i in range(len(agent_init_dicts)):
                                test_set_agent_dict = {'path': agent_init_dicts[i]['path'], 
                                                   'curr_lane_id_order': env.agents_[str(i+1)].closest_lane_id_order_, 
                                                   'curr_waypoint_idx': env.agents_[str(i+1)].closest_waypoint_idx_,
                                                   'agg_level': env.agents_[str(i+1)].agg_level_,
                                                   'v_desired': env.agents_[str(i+1)].v_desired_,
                                                   'b1': env.agents_[str(i+1)].b1_}
                                test_set_agent_dict_list.append(test_set_agent_dict)

                            test_set_init_dicts.append({'ego': test_set_ego_dict, 'agents': test_set_agent_dict_list, 
                                                        'env_config': copy.deepcopy(env_config), 'setting': 3})


intersection_point 0...
intersection_point 1...
intersection_point 2...
intersection_point 3...
permutation=(0, 1, 2)
a_b1: -0.4222003865472924, a_v_desired: 7.160058956322311
b_b1: -0.7911966227237499, b_v_desired: 6.163769118645876
c_b1: -0.4226765219005494, c_v_desired: 7.158773390868517
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
c_b1: 0.16054150414639118, c_v_desired: 8.733462061195256
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
c_b1: 0.6411743835960864, c_v_desired: 10.031170835709434
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
b_b1: 0.3168222015563223, b_v_desired: 9.155419944202071
c_b1: -0.8301028685342852, c_v_desired: 6.058722254957431
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
c_b1: 0.1

c_arrival_noise: 4
c_b1: 0.06499057254891044, c_v_desired: 8.47547454588206
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
c_b1: 0.47894784539941876, c_v_desired: 9.593159182578432
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
b_b1: 0.8251775362807974, b_v_desired: 10.527979347958153
c_b1: -0.8704760322485785, c_v_desired: 5.949714712928839
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
c_b1: 0.14184463430674643, c_v_desired: 8.682980512628216
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
c_b1: 0.6079097475011939, c_v_desired: 9.941356318253224
b_arrival_noise: -3
c_arrival_noise: -4
c_arrival_noise: 4
b_arrival_noise: 4
c_arrival_noise: -4
c_arrival_noise: 4
a_b1: 0.7129482161790255, a_v_desired: 10.22496018

In [62]:
len(test_set_init_dicts)

1332

## Convert dict to agents

In [63]:
# Convert dict to agents
test_set_agent_inits = []

for idx in range(len(test_set_init_dicts)):
    test_set_init_dict = test_set_init_dicts[idx]
    agent_dict = env.convert_dict_to_agents(test_set_init_dict['ego'], test_set_init_dict['agents'])
    # interacting_agents of basic episodes are basically all other agents
    interacting_agents = []
    for i in range(1, len(test_set_init_dict['agents'])+1):
        interacting_agents.append(i)
    test_set_agent_inits.append({'ep_id': 'basic_' + str(idx),
                                 'env_config': test_set_init_dict['env_config'],
                                 'setting': test_set_init_dict['setting'],
                                 'agents_init': agent_dict,
                                 'interacting_agents': interacting_agents})

save_obj(test_set_agent_inits, f'{date}_neighborhood_v4_interaction_set_basic')


# Interaction Set without Arrival Perturbation

In [70]:
env = gym.make('Neighborhood-v4')
env.set_env_config(env_config)
assert env_config['max_num_other_agents'] == 25
env.set_train_config_and_device(train_configs, device)
env.log_name_ = 'out/interaction_set_creation.log'

test_set_init_dicts = []

## Setting 1

In [71]:
# == Setting 1 ==
w = 2.7
b = 8.3
# 0601: change to evenly divided
a_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
b_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
agent_arrival_noise = [0]
ego_pred_depth = 2
agent_pred_depth = 3

# [{'ego': ego_init_dict, 
#   'agents': [agent_init_dict]}]
env_config['num_other_agents'] = 1

# 14 * 3 * 3 = 126
for p in range(len(intersection_points)):
    intersection_point_entry = intersection_points[p]
    print(f"intersection_point {p}...")
    # deciding who's A, who's B
    if len(intersection_point_entry) == 1:
        continue
    elif len(intersection_point_entry) == 2:
        permutations = [(0,1),(1,0)]
    elif len(intersection_point_entry) == 3:
        permutations = [(0,1),(1,0),(1,2),(2,1),(0,2),(2,0)]
        
    for permutation in permutations:
        print(f"permutation={permutation}")
        for b_b1_interval in b_b1_intervals:
            b_b1 = np.random.random() * (b_b1_interval[1] - b_b1_interval[0]) + b_b1_interval[0]
            b_v_desired = w * b_b1 + b
            print(f"b_b1: {b_b1}, b_v_desired: {b_v_desired}")
            for a_b1_interval in a_b1_intervals:
                a_b1 = np.random.random() * (a_b1_interval[1] - a_b1_interval[0]) + a_b1_interval[0]
                a_v_desired = w * a_b1 + b
                print(f"a_b1: {a_b1}, a_v_desired: {a_v_desired}")
                for arrival_noise in agent_arrival_noise:
                    print(f"arrival_noise: {arrival_noise}")
                    ego_path = [intersection_point_entry[permutation[0]][0]]
                    # ego_path.insert(0, env.lane_segments_[intersection_point_entry[permutation[0]][0]].predecessors[0])
                    for _ in range(ego_pred_depth):
                        ego_path.insert(0, env.lane_segments_[ego_path[0]].predecessors[0])
                    ego_path.append(env.lane_segments_[intersection_point_entry[permutation[0]][0]].successors[0])
                    ego_path.append(env.lane_segments_[ego_path[-1]].successors[0]) # ed=2
                    ego_init_dict = {'path': ego_path, 'curr_lane_id_order': ego_pred_depth, 
                                     'curr_waypoint_idx': intersection_point_entry[permutation[0]][1],
                                     'v_desired': a_v_desired, 'b1': a_b1}

                    agent_path = [intersection_point_entry[permutation[1]][0]]
                    # add (agent_pred_depth)-degree predecessors to agent_path to ensure there's enough space to retreat to
                    for d in range(agent_pred_depth):
                        # find the longest predecessor
                        preds = env.lane_segments_[agent_path[0]].predecessors
                        longest_pred_idx = 0
                        if len(preds) > 1:
                            longest_pred_idx = -1
                            longest_pred_length = -float('Inf')
                            for j in range(len(preds)):
                                if len(env.lane_segments_[preds[j]].centerline) > longest_pred_length:
                                    longest_pred_idx = j
                                    longest_pred_length = len(env.lane_segments_[preds[j]].centerline)
                        agent_path.insert(0, env.lane_segments_[agent_path[0]].predecessors[longest_pred_idx])

                    agent_path.append(env.lane_segments_[intersection_point_entry[permutation[1]][0]].successors[0])
                    agent_extending_path = env.generate_agent_extending_path(agent_path[-1]) # make sure agent keeps going
                    agent_path = agent_path + agent_extending_path[1:]
                    agent_init_dicts = [{'path': agent_path, 'curr_lane_id_order': agent_pred_depth, 
                                         'curr_waypoint_idx': intersection_point_entry[permutation[1]][1], 
                                         'v_desired': b_v_desired, 'b1': b_b1}]
                    agents_dict = env.convert_dict_to_agents(ego_init_dict, agent_init_dicts)
                    assert len(agent_init_dicts) == env_config['num_other_agents']
                    state = env.reset(use_saved_agents=agents_dict)

                    cnt = 0
                    target_cnt = 24
                    while (cnt < target_cnt):
                        env.agents_['0'].step(-1, env.dt_)
                        env.agents_['1'].force_action(-1, env.dt_)
                        cnt += 1
                        if (env.agents_['1'].closest_lane_id_order_ == 0 and env.agents_['1'].closest_waypoint_idx_ == 0):
                            raise Exception("agent 1 pushed out of path")
                        if (env.agents_['0'].closest_lane_id_order_ == 0 and env.agents_['0'].closest_waypoint_idx_ == 0):
                            raise Exception("agent 0 pushed out of path")


                    # agent arrival noise
                    if arrival_noise < 0:
                        for t in range(abs(arrival_noise)):
                            env.agents_['1'].force_action(-1, env.dt_)
                            if (env.agents_['1'].closest_lane_id_order_ == 0 and env.agents_['1'].closest_waypoint_idx_ == 0):
                                raise Exception("agent 1 pushed out of path")
                    else:
                        for t in range(arrival_noise):
                            env.agents_['1'].force_action(1, env.dt_)

                    test_set_ego_dict = {'path': ego_init_dict['path'], 
                                         'curr_lane_id_order': env.agents_['0'].closest_lane_id_order_, 
                                         'curr_waypoint_idx': env.agents_['0'].closest_waypoint_idx_,
                                         'v_desired': env.agents_['0'].v_desired_, 'b1': env.agents_['0'].b1_}
                    test_set_agent_dict = {'path': agent_init_dicts[0]['path'], 
                                           'curr_lane_id_order': env.agents_['1'].closest_lane_id_order_, 
                                           'curr_waypoint_idx': env.agents_['1'].closest_waypoint_idx_,
                                           'v_desired': env.agents_['1'].v_desired_, 'b1': env.agents_['1'].b1_}
                    test_set_init_dicts.append({'ego': test_set_ego_dict, 
                                                'agents': [test_set_agent_dict], 
                                                'env_config': copy.deepcopy(env_config),
                                                'setting': 1})


intersection_point 0...
permutation=(0, 1)
b_b1: -0.39039078827668516, b_v_desired: 7.24594487165295
a_b1: -0.7458604909441171, a_v_desired: 6.286176674450884
arrival_noise: 0
a_b1: -0.08374594638474886, a_v_desired: 8.073885944761178
arrival_noise: 0
a_b1: 0.4984553894036508, a_v_desired: 9.645829551389857
arrival_noise: 0
b_b1: 0.016812058791240425, b_v_desired: 8.34539255873635
a_b1: -0.7296108226794967, a_v_desired: 6.33005077876536
arrival_noise: 0
a_b1: -0.07892183664198649, a_v_desired: 8.086911041066637
arrival_noise: 0
a_b1: 0.9616727841446273, a_v_desired: 10.896516517190495
arrival_noise: 0
b_b1: 0.9811519559650081, b_v_desired: 10.949110281105522
a_b1: -0.8034464878501892, a_v_desired: 6.13069448280449
arrival_noise: 0
a_b1: -0.08065558530091205, a_v_desired: 8.082229919687538
arrival_noise: 0
a_b1: 0.8416821041868832, a_v_desired: 10.572541681304585
arrival_noise: 0
permutation=(1, 0)
b_b1: -0.38408724504354586, b_v_desired: 7.262964438382427
a_b1: -0.3728065827981053, a_v

intersection_point 5...
permutation=(0, 1)
b_b1: -0.5273830530490972, b_v_desired: 6.876065756767439
a_b1: -0.9111593242430551, a_v_desired: 5.839869824543752
arrival_noise: 0
a_b1: -0.0431052443369867, a_v_desired: 8.183615840290136
arrival_noise: 0
a_b1: 0.9127881336210173, a_v_desired: 10.764527960776748
arrival_noise: 0
b_b1: -0.007548568644355769, b_v_desired: 8.27961886466024
a_b1: -0.5957090294430826, a_v_desired: 6.691585620503678
arrival_noise: 0
a_b1: 0.07576600732816591, a_v_desired: 8.504568219786048
arrival_noise: 0
a_b1: 0.5404986990503273, a_v_desired: 9.759346487435884
arrival_noise: 0
b_b1: 0.8670179161440239, b_v_desired: 10.640948373588866
a_b1: -0.4348133323840072, a_v_desired: 7.126004002563182
arrival_noise: 0
a_b1: -0.22861746185594667, a_v_desired: 7.682732852988945
arrival_noise: 0
a_b1: 0.531325033169255, a_v_desired: 9.73457758955699
arrival_noise: 0
permutation=(1, 0)
b_b1: -0.7163331382319729, b_v_desired: 6.3659005267736735
a_b1: -0.5763845556607279, a_v_d

In [72]:
print(len(test_set_init_dicts))

126


## Setting 2

In [73]:
# == Setting 2 ==
w = 2.7
b = 8.3
a_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
b_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
c_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
agent_arrival_noise = [0]
ego_pred_depth = 2
agent_pred_depth = 5

env_config['num_other_agents'] = 2

# 14 * 3 * 3 * 3 = 378
for p in range(len(intersection_points)):
    intersection_point_entry = intersection_points[p]
    print(f"intersection_point {p}...")
    # deciding who's A, who's B
    if len(intersection_point_entry) == 1:
        continue
    elif len(intersection_point_entry) == 2:
        permutations = [(0,1),(1,0)]
    elif len(intersection_point_entry) == 3:
        permutations = [(0,1),(1,0),(1,2),(2,1),(0,2),(2,0)]
    
    for permutation in permutations:
        print(f"permutation={permutation}")
        for a_b1_interval in a_b1_intervals:
            a_b1 = np.random.random() * (a_b1_interval[1] - a_b1_interval[0]) + a_b1_interval[0]
            a_v_desired = w * a_b1 + b
            print(f"a_b1: {a_b1}, a_v_desired: {a_v_desired}")
            
            for b_b1_interval in b_b1_intervals:
                b_b1 = np.random.random() * (b_b1_interval[1] - b_b1_interval[0]) + b_b1_interval[0]
                b_v_desired = w * b_b1 + b
                print(f"b_b1: {b_b1}, b_v_desired: {b_v_desired}")
            
                for c_b1_interval in c_b1_intervals:
                    c_b1 = np.random.random() * (c_b1_interval[1] - c_b1_interval[0]) + c_b1_interval[0]
                    c_v_desired = w * c_b1 + b
                    print(f"c_b1: {c_b1}, c_v_desired: {c_v_desired}")
                
                    for arrival_noise in agent_arrival_noise:
                        print(f"arrival_noise: {arrival_noise}")
                        ego_path = [intersection_point_entry[permutation[0]][0]]
                        for _ in range(ego_pred_depth):
                            ego_path.insert(0, env.lane_segments_[ego_path[0]].predecessors[0])
                        ego_path.append(env.lane_segments_[intersection_point_entry[permutation[0]][0]].successors[0])
                        ego_path.append(env.lane_segments_[ego_path[-1]].successors[0]) # ed=2
                        ego_init_dict = {'path': ego_path, 'curr_lane_id_order': ego_pred_depth, 
                                         'curr_waypoint_idx': intersection_point_entry[permutation[0]][1],
                                         'v_desired': a_v_desired, 'b1': a_b1}

                        agent_path = [intersection_point_entry[permutation[1]][0]]
                        # add 4-degree predecessors to agent_path to ensure there's enough space to retreat to
                        for d in range(agent_pred_depth):
                            # find the longest predecessor
                            preds = env.lane_segments_[agent_path[0]].predecessors
                            longest_pred_idx = 0
                            if len(preds) > 1:
                                longest_pred_idx = -1
                                longest_pred_length = -float('Inf')
                                for j in range(len(preds)):
                                    if len(env.lane_segments_[preds[j]].centerline) > longest_pred_length:
                                        longest_pred_idx = j
                                        longest_pred_length = len(env.lane_segments_[preds[j]].centerline)
                            agent_path.insert(0, env.lane_segments_[agent_path[0]].predecessors[longest_pred_idx])

                        agent_path.append(env.lane_segments_[intersection_point_entry[permutation[1]][0]].successors[0])
                        agent_extending_path = env.generate_agent_extending_path(agent_path[-1]) # make sure agent keeps going
                        agent_path = agent_path + agent_extending_path[1:]
                        agent_init_dicts = [{'path': agent_path, 'curr_lane_id_order': agent_pred_depth, 
                                             'curr_waypoint_idx': intersection_point_entry[permutation[1]][1], 
                                             'v_desired': b_v_desired, 'b1': b_b1},
                                            {'path': agent_path, 'curr_lane_id_order': agent_pred_depth, 
                                             'curr_waypoint_idx': intersection_point_entry[permutation[1]][1], 
                                             'v_desired': c_v_desired, 'b1': c_b1}]
                        agents_dict = env.convert_dict_to_agents(ego_init_dict, agent_init_dicts)
                        assert len(agent_init_dicts) == env_config['num_other_agents']
                        state = env.reset(use_saved_agents=agents_dict)

                        cnt = 0
                        target_cnt = 24
                        while (cnt < target_cnt):
                            env.agents_['0'].step(-1, env.dt_)
                            env.agents_['1'].force_action(-1, env.dt_)
                            cnt += 1
                            if (env.agents_['1'].closest_lane_id_order_ == 0 and env.agents_['1'].closest_waypoint_idx_ == 0):
                                raise Exception("agent 1 pushed out of path")
                            if (env.agents_['0'].closest_lane_id_order_ == 0 and env.agents_['0'].closest_waypoint_idx_ == 0):
                                raise Exception("agent 0 pushed out of path")

                        # agent arrival noise
                        if arrival_noise < 0:
                            for t in range(abs(arrival_noise)):
                                env.agents_['1'].force_action(-1, env.dt_)
                                if (env.agents_['1'].closest_lane_id_order_ == 0 and env.agents_['1'].closest_waypoint_idx_ == 0):
                                    raise Exception("agent 1 pushed out of path")
                        else:
                            for t in range(arrival_noise):
                                env.agents_['1'].force_action(1, env.dt_)

                        # now find C's position
                        # change C dict to match B location
                        agent_init_dicts[1]['curr_lane_id_order'] = env.agents_['1'].closest_lane_id_order_
                        agent_init_dicts[1]['curr_waypoint_idx'] = env.agents_['1'].closest_waypoint_idx_
                        agent_c = env.convert_dict_to_agent(agent_init_dicts[1], False, '2')
                        env.agents_['2'] = agent_c

                        while not (calculate_poly_distance(env.agents_['1'], env.agents_['2']) >= env.agents_['2'].fad_distance()):
                            env.agents_['2'].force_action(-1, env.dt_)
                            if (env.agents_['2'].closest_lane_id_order_ == 0 and env.agents_['2'].closest_waypoint_idx_ == 0):
                                raise Exception("agent 2 pushed out of path")

                        # save the 3 agents info
                        test_set_ego_dict = {'path': ego_init_dict['path'], 
                                             'curr_lane_id_order': env.agents_['0'].closest_lane_id_order_, 
                                             'curr_waypoint_idx': env.agents_['0'].closest_waypoint_idx_,
                                             'v_desired': env.agents_['0'].v_desired_, 
                                             'b1': env.agents_['0'].b1_}
                        test_set_agent_dict_list = []
                        for i in range(len(agent_init_dicts)):
                            test_set_agent_dict = {'path': agent_init_dicts[i]['path'], 
                                               'curr_lane_id_order': env.agents_[str(i+1)].closest_lane_id_order_, 
                                               'curr_waypoint_idx': env.agents_[str(i+1)].closest_waypoint_idx_,
                                               'agg_level': env.agents_[str(i+1)].agg_level_,
                                               'v_desired': env.agents_[str(i+1)].v_desired_,
                                               'b1': env.agents_[str(i+1)].b1_,}
                            test_set_agent_dict_list.append(test_set_agent_dict)

                        test_set_init_dicts.append({'ego': test_set_ego_dict, 'agents': test_set_agent_dict_list, 
                                                    'env_config': copy.deepcopy(env_config), 'setting': 2})


intersection_point 0...
permutation=(0, 1)
a_b1: -0.9471250852003967, a_v_desired: 5.742762269958929
b_b1: -0.7570256822801311, b_v_desired: 6.256030657843647
c_b1: -0.4129489721671703, c_v_desired: 7.185037775148641
arrival_noise: 0
c_b1: 0.24895578110437305, c_v_desired: 8.972180608981809
arrival_noise: 0
c_b1: 0.3433631542283789, c_v_desired: 9.227080516416624
arrival_noise: 0
b_b1: -0.09251989593172452, b_v_desired: 8.050196280984345
c_b1: -0.3704350582470515, c_v_desired: 7.299825342732961
arrival_noise: 0
c_b1: -0.07851390318652518, c_v_desired: 8.088012461396383
arrival_noise: 0
c_b1: 0.5788560239536549, c_v_desired: 9.86291126467487
arrival_noise: 0
b_b1: 0.7281816602822122, b_v_desired: 10.266090482761975
c_b1: -0.6314054439693236, c_v_desired: 6.595205301282827
arrival_noise: 0
c_b1: 0.2531782221644063, c_v_desired: 8.983581199843897
arrival_noise: 0
c_b1: 0.8965140215336078, c_v_desired: 10.720587858140743
arrival_noise: 0
a_b1: 0.29545931604385417, a_v_desired: 9.0977401533

c_b1: -0.11723559946462891, c_v_desired: 7.983463881445503
arrival_noise: 0
c_b1: 0.9651762999308433, c_v_desired: 10.905976009813278
arrival_noise: 0
b_b1: -0.2793389367352445, b_v_desired: 7.54578487081484
c_b1: -0.754810435520678, c_v_desired: 6.26201182409417
arrival_noise: 0
c_b1: -0.3298136179521557, c_v_desired: 7.409503231529181
arrival_noise: 0
c_b1: 0.5412856078059134, c_v_desired: 9.761471141075967
arrival_noise: 0
b_b1: 0.8829925857653338, b_v_desired: 10.684079981566402
c_b1: -0.5314611637820046, c_v_desired: 6.865054857788588
arrival_noise: 0
c_b1: -0.2627716038514715, c_v_desired: 7.590516669601028
arrival_noise: 0
c_b1: 0.4237198648165984, c_v_desired: 9.444043635004817
arrival_noise: 0
a_b1: 0.09568297700308143, a_v_desired: 8.55834403790832
b_b1: -0.4911242941761279, b_v_desired: 6.973964405724455
c_b1: -0.8915135381850662, c_v_desired: 5.892913446900321
arrival_noise: 0
c_b1: -0.2765041878250303, c_v_desired: 7.553438692872419
arrival_noise: 0
c_b1: 0.534303520740201

c_b1: 0.03689734326470118, c_v_desired: 8.399622826814694
arrival_noise: 0
c_b1: 0.4984715674488114, c_v_desired: 9.645873232111791
arrival_noise: 0
b_b1: -0.18251729165243283, b_v_desired: 7.807203312538432
c_b1: -0.8185232357775978, c_v_desired: 6.089987263400486
arrival_noise: 0
c_b1: -0.3083018589932993, c_v_desired: 7.467584980718092
arrival_noise: 0
c_b1: 0.397381453371215, c_v_desired: 9.372929924102282
arrival_noise: 0
b_b1: 0.9283100367613, b_v_desired: 10.806437099255511
c_b1: -0.6214279673116505, c_v_desired: 6.622144488258544
arrival_noise: 0
c_b1: -0.3027398614163753, c_v_desired: 7.482602374175787
arrival_noise: 0
c_b1: 0.6751431752766516, c_v_desired: 10.12288657324696
arrival_noise: 0
a_b1: 0.044384408285009114, a_v_desired: 8.419837902369526
b_b1: -0.3697121146053064, b_v_desired: 7.301777290565673
c_b1: -0.9649993157260309, c_v_desired: 5.694501847539717
arrival_noise: 0
c_b1: 0.3258579974576326, c_v_desired: 9.179816593135609
arrival_noise: 0
c_b1: 0.4757774873500301

c_b1: 0.17048293532363729, c_v_desired: 8.76030392537382
arrival_noise: 0
c_b1: 0.4072083465165688, c_v_desired: 9.399462535594736
arrival_noise: 0
a_b1: 0.17998671980226205, a_v_desired: 8.785964143466108
b_b1: -0.4242988705885249, b_v_desired: 7.1543930494109835
c_b1: -0.9538704283182962, c_v_desired: 5.724549843540601
arrival_noise: 0
c_b1: -0.12381253688014673, c_v_desired: 7.965706150423604
arrival_noise: 0
c_b1: 0.9235308438900471, c_v_desired: 10.793533278503128
arrival_noise: 0
b_b1: -0.21601565172781795, b_v_desired: 7.716757740334892
c_b1: -0.8412392253090196, c_v_desired: 6.028654091665647
arrival_noise: 0
c_b1: -0.11964215505173145, c_v_desired: 7.976966181360326
arrival_noise: 0
c_b1: 0.8104411382907672, c_v_desired: 10.488191073385073
arrival_noise: 0
b_b1: 0.3824204285714809, b_v_desired: 9.332535157142999
c_b1: -0.8913910235554391, c_v_desired: 5.8932442364003155
arrival_noise: 0
c_b1: -0.19203589862692622, c_v_desired: 7.7815030737073
arrival_noise: 0
c_b1: 0.693234700

c_b1: -0.2491445213117285, c_v_desired: 7.627309792458334
arrival_noise: 0
c_b1: 0.3761938618048882, c_v_desired: 9.315723426873198
arrival_noise: 0
b_b1: -0.2415185466617061, b_v_desired: 7.6478999240133945
c_b1: -0.8986065409246613, c_v_desired: 5.873762339503415
arrival_noise: 0
c_b1: -0.1441059927974759, c_v_desired: 7.910913819446816
arrival_noise: 0
c_b1: 0.7046245752186605, c_v_desired: 10.202486353090384
arrival_noise: 0
b_b1: 0.921041254350494, b_v_desired: 10.786811386746335
c_b1: -0.8610269990730339, c_v_desired: 5.975227102502808
arrival_noise: 0
c_b1: -0.050670537655723935, c_v_desired: 8.163189548329546
arrival_noise: 0
c_b1: 0.8091380450321373, c_v_desired: 10.484672721586772
arrival_noise: 0
a_b1: 0.875123018596144, a_v_desired: 10.66283215020959
b_b1: -0.8629341172797493, b_v_desired: 5.970077883344677
c_b1: -0.510314523782349, c_v_desired: 6.922150785787658
arrival_noise: 0
c_b1: 0.15727258880460204, c_v_desired: 8.724635989772427
arrival_noise: 0
c_b1: 0.616183994792

In [74]:
print(len(test_set_init_dicts)) # 126 + 378 = 504

504


## Setting 3

In [75]:
# == Setting 3 ==
w = 2.7
b = 8.3
a_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
b_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
c_b1_intervals = [(-1,-0.33),(-0.34,0.33),(0.34,1.0)]
b_arrival_noises = [0]
c_arrival_noises = [0]
ego_pred_depth = 2
agent_pred_depth = 2

env_config['num_other_agents'] = 2

# 1 * 3 * 3 * 3 * 3 * 1 * 1 = 81
for p in range(len(intersection_points)):
    intersection_point_entry = intersection_points[p]
    print(f"intersection_point {p}...")
    # deciding who's A, who's B
    if len(intersection_point_entry) < 3:
        continue
    else:
        permutations = [(0,1,2),(1,0,2),(2,0,1)]
    
    for permutation in permutations:
        print(f"permutation={permutation}")
        for a_b1_interval in a_b1_intervals:
            a_b1 = np.random.random() * (a_b1_interval[1] - a_b1_interval[0]) + a_b1_interval[0]
            a_v_desired = w * a_b1 + b
            print(f"a_b1: {a_b1}, a_v_desired: {a_v_desired}")
            
            for b_b1_interval in b_b1_intervals:
                b_b1 = np.random.random() * (b_b1_interval[1] - b_b1_interval[0]) + b_b1_interval[0]
                b_v_desired = w * b_b1 + b
                print(f"b_b1: {b_b1}, b_v_desired: {b_v_desired}")
            
                for c_b1_interval in c_b1_intervals:
                    c_b1 = np.random.random() * (c_b1_interval[1] - c_b1_interval[0]) + c_b1_interval[0]
                    c_v_desired = w * c_b1 + b
                    print(f"c_b1: {c_b1}, c_v_desired: {c_v_desired}")
                    
                    bc_b1 = [b_b1, c_b1]
                    bc_v_desired = [b_v_desired, c_v_desired]
                    
                    for b_arrival_noise in b_arrival_noises:
                        print(f"b_arrival_noise: {b_arrival_noise}")
                        for c_arrival_noise in c_arrival_noises:
                            print(f"c_arrival_noise: {c_arrival_noise}")
                            curr_arrival_noises = [b_arrival_noise, c_arrival_noise]
                            ego_path = [intersection_point_entry[permutation[0]][0]]
                            for _ in range(ego_pred_depth):
                                ego_path.insert(0, env.lane_segments_[ego_path[0]].predecessors[0])
                            ego_path.append(env.lane_segments_[intersection_point_entry[permutation[0]][0]].successors[0])
                            ego_path.append(env.lane_segments_[ego_path[-1]].successors[0]) # ed=2
                            ego_init_dict = {'path': ego_path, 'curr_lane_id_order': ego_pred_depth, 
                                             'curr_waypoint_idx': intersection_point_entry[permutation[0]][1],
                                             'v_desired': a_v_desired, 'b1': a_b1}

                            agent_init_dicts = []
                            for j in range(1,3):
                                agent_path = [intersection_point_entry[permutation[j]][0]]
                                # add 4-degree predecessors to agent_path to ensure there's enough space to retreat to
                                for d in range(agent_pred_depth):
                                    # find the longest predecessor
                                    preds = env.lane_segments_[agent_path[0]].predecessors
                                    longest_pred_idx = 0
                                    if len(preds) > 1:
                                        longest_pred_idx = -1
                                        longest_pred_length = -float('Inf')
                                        for j in range(len(preds)):
                                            if len(env.lane_segments_[preds[j]].centerline) > longest_pred_length:
                                                longest_pred_idx = j
                                                longest_pred_length = len(env.lane_segments_[preds[j]].centerline)
                                    agent_path.insert(0, env.lane_segments_[agent_path[0]].predecessors[longest_pred_idx])

                                agent_path.append(env.lane_segments_[intersection_point_entry[permutation[j]][0]].successors[0])
                                agent_extending_path = env.generate_agent_extending_path(agent_path[-1]) # make sure agent keeps going
                                agent_path = agent_path + agent_extending_path[1:]
                                agent_init_dicts.append({'path': agent_path, 'curr_lane_id_order': agent_pred_depth, 
                                                         'curr_waypoint_idx': intersection_point_entry[permutation[j]][1], 
                                                         'v_desired': bc_v_desired[j-1], 'b1': bc_b1[j-1]})

                            agents_dict = env.convert_dict_to_agents(ego_init_dict, agent_init_dicts)
                            assert len(agent_init_dicts) == env_config['num_other_agents']
                            state = env.reset(use_saved_agents=agents_dict)

                            cnt = 0
                            target_cnt = 24
                            while (cnt < target_cnt):
                                env.agents_['0'].step(-1, env.dt_)
                                env.agents_['1'].force_action(-1, env.dt_)
                                env.agents_['2'].force_action(-1, env.dt_)
                                cnt += 1
                                # check if any of the agents gets pushed out of path
                                for agent_id in range(env_config['num_other_agents'] + 1):
                                    agent_id_str = str(agent_id)
                                    if (env.agents_[agent_id_str].closest_lane_id_order_ == 0 and env.agents_[agent_id_str].closest_waypoint_idx_ == 0):
                                        raise Exception(f"agent {agent_id_str} pushed out of path")

                            # agent arrival noise
                            for j in range(1,3):
                                j_str = str(j)
                                if curr_arrival_noises[j-1] < 0:
                                    for t in range(abs(curr_arrival_noises[j-1])):
                                        env.agents_[j_str].force_action(-1, env.dt_)
                                        if (env.agents_[j_str].closest_lane_id_order_ == 0 and env.agents_[j_str].closest_waypoint_idx_ == 0):
                                            raise Exception(f"agent {j_str} pushed out of path")
                                else:
                                    for t in range(curr_arrival_noises[j-1]):
                                        env.agents_[j_str].force_action(1, env.dt_)

                            # save the 3 agents info
                            test_set_ego_dict = {'path': ego_init_dict['path'], 
                                                 'curr_lane_id_order': env.agents_['0'].closest_lane_id_order_, 
                                                 'curr_waypoint_idx': env.agents_['0'].closest_waypoint_idx_,
                                                 'v_desired': env.agents_['0'].v_desired_,
                                                 'b1': env.agents_['0'].b1_}
                            test_set_agent_dict_list = []
                            for i in range(len(agent_init_dicts)):
                                test_set_agent_dict = {'path': agent_init_dicts[i]['path'], 
                                                   'curr_lane_id_order': env.agents_[str(i+1)].closest_lane_id_order_, 
                                                   'curr_waypoint_idx': env.agents_[str(i+1)].closest_waypoint_idx_,
                                                   'agg_level': env.agents_[str(i+1)].agg_level_,
                                                   'v_desired': env.agents_[str(i+1)].v_desired_,
                                                   'b1': env.agents_[str(i+1)].b1_}
                                test_set_agent_dict_list.append(test_set_agent_dict)

                            test_set_init_dicts.append({'ego': test_set_ego_dict, 'agents': test_set_agent_dict_list, 
                                                        'env_config': copy.deepcopy(env_config), 'setting': 3})


intersection_point 0...
intersection_point 1...
intersection_point 2...
intersection_point 3...
permutation=(0, 1, 2)
a_b1: -0.9385303351173264, a_v_desired: 5.765968095183219
b_b1: -0.3380673509385288, b_v_desired: 7.387218152465973
c_b1: -0.8963212278160183, c_v_desired: 5.879932684896751
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: -0.16782699462342363, c_v_desired: 7.846867114516757
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: 0.8743707777767644, c_v_desired: 10.660801099997265
b_arrival_noise: 0
c_arrival_noise: 0
b_b1: -0.13460256297355808, b_v_desired: 7.936573079971394
c_b1: -0.5848160737827754, c_v_desired: 6.720996600786507
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: 0.24172896725965415, c_v_desired: 8.952668211601067
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: 0.8027997129750646, c_v_desired: 10.467559225032675
b_arrival_noise: 0
c_arrival_noise: 0
b_b1: 0.3903053710637968, b_v_desired: 9.353824501872252
c_b1: -0.5354065111458076, c_v_desired: 6.85440241990632
b_arrival_noise:

b_b1: -0.13465800804763312, b_v_desired: 7.9364233782713915
c_b1: -0.597874638320806, c_v_desired: 6.685738476533825
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: -0.03608285834213326, c_v_desired: 8.20257628247624
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: 0.7232655579286467, c_v_desired: 10.252817006407346
b_arrival_noise: 0
c_arrival_noise: 0
b_b1: 0.8149469587195466, b_v_desired: 10.500356788542776
c_b1: -0.7617457891049646, c_v_desired: 6.243286369416596
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: -0.1183781866390198, c_v_desired: 7.980378896074647
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: 0.4885690520783519, c_v_desired: 9.619136440611552
b_arrival_noise: 0
c_arrival_noise: 0
a_b1: 0.488771314238443, a_v_desired: 9.619682548443796
b_b1: -0.9500900745307961, b_v_desired: 5.734756798766851
c_b1: -0.33070906229016284, c_v_desired: 7.407085531816561
b_arrival_noise: 0
c_arrival_noise: 0
c_b1: -0.12498720184633019, c_v_desired: 7.962534555014909
b_arrival_noise: 0
c_arrival_noise: 0


In [76]:
print(len(test_set_init_dicts))

585


## Convert dict to agents

In [77]:
# Convert dict to agents
test_set_agent_inits = []

for idx in range(len(test_set_init_dicts)):
    test_set_init_dict = test_set_init_dicts[idx]
    agent_dict = env.convert_dict_to_agents(test_set_init_dict['ego'], test_set_init_dict['agents'])
    # interacting_agents of basic episodes are basically all other agents
    interacting_agents = []
    for i in range(1, len(test_set_init_dict['agents'])+1):
        interacting_agents.append(i)
    test_set_agent_inits.append({'ep_id': 'ess_' + str(idx),
                                 'env_config': test_set_init_dict['env_config'],
                                 'setting': test_set_init_dict['setting'],
                                 'agents_init': agent_dict,
                                 'interacting_agents': interacting_agents})

save_obj(test_set_agent_inits, f'{date}_neighborhood_v4_interaction_set_essential')


# Create training_set_interaction, testing_set_interaction, eval_set_interaction

In [101]:
def add_agents_to_episode(curr_ep_dict):
    env.set_env_config(curr_ep_dict['env_config'])
    env.reset(use_saved_agents=curr_ep_dict['agents_init'])
    
    num_agents_to_add = np.random.randint(low=5,high=21)
    temp_agents = env.generate_other_agents(curr_ep_dict['agents_init']['0'].path_, num_agents_to_add)

    # new env_config and agents_init
    expanded_env_config = copy.deepcopy(curr_ep_dict['env_config'])
    expanded_env_config['num_other_agents'] += num_agents_to_add
    expanded_env_config['max_num_other_agents'] = 25
    assert expanded_env_config['num_other_agents'] <= expanded_env_config['max_num_other_agents']
    
    expanded_agents_init = copy.deepcopy(curr_ep_dict['agents_init'])
    expanded_agents_init.update(temp_agents)

    # save
    expanded_ep_dict = {'ep_id': curr_ep_dict['ep_id'] + "_addon",
                                          'setting': copy.deepcopy(curr_ep_dict['setting']),
                                          'env_config':expanded_env_config,
                                          'agents_init':expanded_agents_init,
                                          'interacting_agents': copy.deepcopy(curr_ep_dict['interacting_agents'])}
    return expanded_ep_dict, num_agents_to_add
    

In [103]:
# interaction sets are all lists
# set stats are dictionaries indexed by ep_ids, which come from interaction sets
training_set_interaction = []
testing_set_interaction = []
eval_set_interaction = []

# (start idx, end idx (excl.), training_num_to_pick, eval_num_to_pick_addon, eval_num_to_pick_non_addon)
quantities_to_pick = {'basic'     : [(0,252,202,7,6), (252,1008,605,20,19), (1008,1332,260,9,9)],
                      'essential' : [(0,126,101,3,3), (126,504,303,10,9), (504,585,65,3,2)]}
basic_interaction_set =  load_obj(f'{date}_neighborhood_v4_interaction_set_basic')
essential_interaction_set =  load_obj(f'{date}_neighborhood_v4_interaction_set_essential')

env = gym.make('Neighborhood-v4')
env.set_env_config(env_config)
assert env_config['max_num_other_agents'] == 25
env.set_train_config_and_device(train_configs, device)
env.log_name_ = 'out/interaction_set_creation.log'

In [104]:
for set_type in quantities_to_pick.keys():
    print(set_type)
    # pick an interaction set to work with
    if set_type == 'basic':
        curr_set = basic_interaction_set
    elif set_type == 'essential':
        curr_set = essential_interaction_set
    else:
        raise Exception("Invalid set_type")
    
    # for each range_entry in quantities_to_pick:
    for range_entry in quantities_to_pick[set_type]:
        print(f"starting range_entry: {range_entry}")
        # randomly shuffle indices between start and end idx
        idx_range = np.arange(range_entry[0], range_entry[1])
        np.random.shuffle(idx_range)
        # for the first training_num_to_pick elements in the shuffled indices
        training_num_to_pick = range_entry[2]
        cnt = 0
        for training_idx in idx_range[:training_num_to_pick]:
            # log
            if cnt % 10 ==  9:
                print(f"Training cnt {cnt} | idx {training_idx}")
            # addon for the first 50% of them and add to training_set_interaction
            if cnt < int(0.5 * training_num_to_pick):
                expanded_ep_dict, num_agents_to_add = add_agents_to_episode(curr_set[training_idx])
                training_set_interaction.append(expanded_ep_dict)
                # log
                if cnt % 10 == 9:
                    print(f"Training cnt {cnt} | idx {training_idx} | added {num_agents_to_add} agents")
            else:
                training_set_interaction.append(copy.deepcopy(curr_set[training_idx]))
            cnt += 1
        # for the rest
        testing_num_to_pick = range_entry[1] - range_entry[0] - training_num_to_pick
        cnt = 0
        eval_addon_cnt = 0
        eval_non_addon_cnt = 0
        for testing_idx in idx_range[training_num_to_pick:]:
            # log
            if cnt % 10 == 9:
                print(f"Testing cnt {cnt} | idx {training_idx}")
            # addon for the first 50% of them and add to testing_set_interaction
            # pick corresponding numbers from addon and non-addon episodes and add to eval_set_interaction
            if cnt < int(0.5 * testing_num_to_pick):
                expanded_ep_dict, num_agents_to_add = add_agents_to_episode(curr_set[testing_idx])
                testing_set_interaction.append(expanded_ep_dict)
                # log
                if cnt % 10 == 9:
                    print(f"Testing cnt {cnt} | idx {training_idx} | added {num_agents_to_add} agents")
                if eval_addon_cnt < range_entry[3]:
                    eval_set_interaction.append(expanded_ep_dict)
                    eval_addon_cnt += 1
            else:
                testing_set_interaction.append(copy.deepcopy(curr_set[testing_idx]))
                if eval_non_addon_cnt < range_entry[4]:
                    eval_set_interaction.append(copy.deepcopy(curr_set[testing_idx]))
                    eval_non_addon_cnt += 1
            cnt += 1
            
print(len(training_set_interaction), len(testing_set_interaction), len(eval_set_interaction))
        
    

basic
starting range_entry: (0, 252, 202, 7, 6)
Training cnt 9 | idx 22
Training cnt 9 | idx 22 | added 14 agents
Training cnt 19 | idx 88
Training cnt 19 | idx 88 | added 8 agents
Training cnt 29 | idx 2
Training cnt 29 | idx 2 | added 7 agents
Training cnt 39 | idx 83
Training cnt 39 | idx 83 | added 5 agents
Training cnt 49 | idx 125
Training cnt 49 | idx 125 | added 10 agents
Training cnt 59 | idx 60
Training cnt 59 | idx 60 | added 20 agents
Training cnt 69 | idx 30
Training cnt 69 | idx 30 | added 15 agents
Training cnt 79 | idx 177
Training cnt 79 | idx 177 | added 9 agents
Training cnt 89 | idx 41
Training cnt 89 | idx 41 | added 9 agents
Training cnt 99 | idx 191
Training cnt 99 | idx 191 | added 20 agents
Training cnt 109 | idx 207
Training cnt 119 | idx 69
Training cnt 129 | idx 218
Training cnt 139 | idx 68
Training cnt 149 | idx 161
Training cnt 159 | idx 43
Training cnt 169 | idx 3
Training cnt 179 | idx 236
Training cnt 189 | idx 37
Training cnt 199 | idx 233
Testing cnt

Training cnt 209 | idx 165
Training cnt 219 | idx 400
Training cnt 229 | idx 223
Training cnt 239 | idx 398
Training cnt 249 | idx 330
Training cnt 259 | idx 216
Training cnt 269 | idx 138
Training cnt 279 | idx 426
Training cnt 289 | idx 231
Training cnt 299 | idx 383
Testing cnt 9 | idx 482
Testing cnt 9 | idx 482 | added 20 agents
Testing cnt 19 | idx 482
Testing cnt 19 | idx 482 | added 8 agents
Testing cnt 29 | idx 482
Testing cnt 29 | idx 482 | added 15 agents
Testing cnt 39 | idx 482
Testing cnt 49 | idx 482
Testing cnt 59 | idx 482
Testing cnt 69 | idx 482
starting range_entry: (504, 585, 65, 3, 2)
Training cnt 9 | idx 569
Training cnt 9 | idx 569 | added 20 agents
Training cnt 19 | idx 581
Training cnt 19 | idx 581 | added 10 agents
Training cnt 29 | idx 510
Training cnt 29 | idx 510 | added 15 agents
Training cnt 39 | idx 572
Training cnt 49 | idx 557
Training cnt 59 | idx 504
Testing cnt 9 | idx 538
1536 381 100


In [105]:
# Save all three sets
save_obj(training_set_interaction, f'{date}_training_set_interaction')
save_obj(testing_set_interaction, f'{date}_testing_set_interaction')
save_obj(eval_set_interaction, f'{date}_eval_set_interaction')
